In [1]:
!pip install --upgrade pymongo certifi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.1 MB/s eta 0:00:00


In [3]:

# Install pymongo for MongoDB connection


# Import necessary libraries
from pymongo import MongoClient
import pprint

# Replace with your MongoDB Atlas connection string
connection_string = "mongodb+srv://<db_username>:<db_password>@cluster0.88cwn.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Connect to MongoDB Atlas
client = MongoClient(connection_string)

# Access the sample_mflix database and the movies collection
db = client['sample_mflix']
collection = db['movies']

In [4]:
#Exercise 1. Basic Searching and Filtering

#1: Write a query to find the first movie that has the genre "Action"
action_movie = collection.find_one({"genres": "Action"}, {"_id": 0, "title": 1})
print(action_movie)




{'title': 'From Hand to Mouth'}


In [7]:
#2. Write a query to find all movies released after the year 2000 (Return the first 5 results).
movies_after_2000 = collection.find(
    {"year": {"$gt": 2000}}, {"_id": 0, "title": 1}).limit(5)
for movie in movies_after_2000:
    print(movie)


{'title': 'Kate & Leopold'}
{'title': 'Glitter'}
{'title': 'Shrek'}
{'title': 'Heartbreakers'}
{'title': 'From Hell'}


In [8]:
#3. Write a query to find all movies where the IMDb rating is greater than 8.5 (Return the first 5 results).
high_rated_movies = collection.find(
    {"imdb.rating": {"$gt": 8.5}},
    {"_id": 0, "title": 1, "imdb.rating": 1} ).limit(5)

for movie in high_rated_movies:
    print(movie)

sorted_drama_movies = collection.find({"genres": "Drama"}).sort("year", 1).limit(5)

{'imdb': {'rating': 8.6}, 'title': 'Modern Times'}
{'title': 'City Lights', 'imdb': {'rating': 8.6}}
{'title': "It's a Wonderful Life", 'imdb': {'rating': 8.6}}
{'title': 'Rear Window', 'imdb': {'rating': 8.6}}
{'title': 'Casablanca', 'imdb': {'rating': 8.6}}


In [9]:
#4. Write a query to find all movies where the genre contains both "Action" and "Adventure".
action_adventure_movies = collection.find(
    {"genres": {"$all": ["Action", "Adventure"]}},
    {"_id": 0, "title": 1, "genres": 1}
)

for movie in action_adventure_movies:
    print(movie)

{'genres': ['Action', 'Adventure', 'Sci-Fi'], 'title': 'Flash Gordon'}
{'genres': ['Action', 'Adventure', 'Romance'], 'title': 'The Charge of the Light Brigade'}
{'genres': ['Action', 'Adventure', 'Drama'], 'title': 'North West Mounted Police'}
{'genres': ['Action', 'Adventure', 'Romance'], 'title': 'Montana'}
{'genres': ['Adventure', 'Action'], 'title': 'The Black Pirate'}
{'genres': ['Action', 'Adventure', 'Drama'], 'title': 'Across the Pacific'}
{'genres': ['Action', 'Adventure', 'Comedy'], 'title': 'Loose in London'}
{'genres': ['Action', 'Adventure', 'Romance'], 'title': 'The Adventures of Robin Hood'}
{'genres': ['Action', 'Adventure', 'Romance'], 'title': 'The Mark of Zorro'}
{'genres': ['Action', 'Adventure', 'Family'], 'title': 'Jungle Book'}
{'genres': ['Adventure', 'Romance', 'Action'], 'title': "King Solomon's Mines"}
{'genres': ['Action', 'Adventure', 'Drama'], 'title': 'Knights of the Round Table'}
{'genres': ['Action', 'Adventure', 'Family'], 'title': 'The Great Locomoti

In [10]:
#Exercise 2: Sorting Results

#1. Write a query to find all movies where the genre is "Comedy" and sort them by IMDb rating in descending order (Return the first 5 results).
comedy_movies = collection.find(
    {"genres": "Comedy", "imdb.rating": {"$ne": ""}},
    {"_id": 0, "title": 1, "imdb.rating": 1} ).sort("imdb.rating", -1).limit(5)

for movie in comedy_movies:
    print(movie)

{'title': 'Over the Garden Wall', 'imdb': {'rating': 9.2}}
{'title': 'The Chaos Class Failed the Class', 'imdb': {'rating': 9.0}}
{'title': 'The Marathon Family', 'imdb': {'rating': 9.0}}
{'title': 'Heart of a Dog', 'imdb': {'rating': 9.0}}
{'title': 'Balkan Spy', 'imdb': {'rating': 8.9}}


In [12]:
#2. Write a query to find all movies where the genre is "Drama" and sort them by release year in ascending order (Return the first 5 results).
drama_movies = collection.find(
    {"genres": "Drama"},
    {"_id": 0, "title": 1, "year": 1, "genres": 1} ).sort("year", 1).limit(5)

for movie in drama_movies:
    print(movie)


{'genres': ['Short', 'Drama'], 'title': 'A Corner in Wheat', 'year': 1909}
{'genres': ['Crime', 'Drama'], 'title': 'Traffic in Souls', 'year': 1913}
{'genres': ['Drama', 'History'], 'title': 'In the Land of the Head Hunters', 'year': 1914}
{'genres': ['Drama'], 'title': 'The Italian', 'year': 1915}
{'genres': ['Biography', 'Crime', 'Drama'], 'title': 'Regeneration', 'year': 1915}


In [13]:
#Exercise 3: Aggregation Pipeline

#1. Write an aggregation pipeline that calculates the average IMDb rating for each genre (Return the top 5 genres).
avg_rating_by_genre = collection.aggregate([
{"$unwind": "$genres"},
{"$group": {"_id": "$genres", "avg_rating": {"$avg": "$imdb.rating"}}},
{"$sort": {"avg_rating": -1}},
{"$limit": 5} ])
for genre in avg_rating_by_genre:
    print(genre)





{'_id': 'Film-Noir', 'avg_rating': 7.397402597402598}
{'_id': 'Short', 'avg_rating': 7.377574370709382}
{'_id': 'Documentary', 'avg_rating': 7.365679824561403}
{'_id': 'News', 'avg_rating': 7.252272727272728}
{'_id': 'History', 'avg_rating': 7.1696100917431185}


In [14]:
#2. Write an aggregation pipeline to find the top 5 directors by the average IMDb rating of their movies.
top_directors = collection.aggregate([
    {"$unwind": "$directors"},
    {"$group": {"_id": "$directors", "avg_rating": {"$avg": "$imdb.rating"}}},
    {"$sort": {"avg_rating": -1}},
    {"$limit": 5} ])
for director in top_directors:
    print(director)



{'_id': 'Sara Hirsh Bordo', 'avg_rating': 9.4}
{'_id': 'Kevin Derek', 'avg_rating': 9.3}
{'_id': 'Michael Benson', 'avg_rating': 9.0}
{'_id': 'Slobodan Sijan', 'avg_rating': 8.95}
{'_id': 'Sundar C.', 'avg_rating': 8.9}


In [15]:
#3. Write an aggregation pipeline to calculate the total number of movies released in each year (Sort the results by the year).
movies_per_year = collection.aggregate([
    {"$group": {"_id": "$year", "total_movies": {"$sum": 1}}},
    {"$sort": {"_id": 1}} ])
for year in movies_per_year:
    print(year)

{'_id': 1896, 'total_movies': 2}
{'_id': 1903, 'total_movies': 1}
{'_id': 1909, 'total_movies': 1}
{'_id': 1911, 'total_movies': 2}
{'_id': 1913, 'total_movies': 1}
{'_id': 1914, 'total_movies': 3}
{'_id': 1915, 'total_movies': 2}
{'_id': 1916, 'total_movies': 2}
{'_id': 1917, 'total_movies': 2}
{'_id': 1918, 'total_movies': 1}
{'_id': 1919, 'total_movies': 1}
{'_id': 1920, 'total_movies': 4}
{'_id': 1921, 'total_movies': 5}
{'_id': 1922, 'total_movies': 3}
{'_id': 1923, 'total_movies': 2}
{'_id': 1924, 'total_movies': 6}
{'_id': 1925, 'total_movies': 3}
{'_id': 1926, 'total_movies': 6}
{'_id': 1927, 'total_movies': 4}
{'_id': 1928, 'total_movies': 8}
{'_id': 1929, 'total_movies': 7}
{'_id': 1930, 'total_movies': 10}
{'_id': 1931, 'total_movies': 20}
{'_id': 1932, 'total_movies': 18}
{'_id': 1933, 'total_movies': 20}
{'_id': 1934, 'total_movies': 23}
{'_id': 1935, 'total_movies': 31}
{'_id': 1936, 'total_movies': 30}
{'_id': 1937, 'total_movies': 31}
{'_id': 1938, 'total_movies': 38}
{

In [16]:
#Exercise 4: Updating and Deleting Documents

#1. Write a query to update the IMDb rating of a movie with the title "The Godfather" to 9.5.
collection.update_one({"title": "The Godfather"}, {"$set": {"imdb.rating": 9.5}})



UpdateResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000121'), 'opTime': {'ts': Timestamp(1728266808, 41), 't': 289}, 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728266808, 41), 'signature': {'hash': b' Kd9\xfe,\xb1\x02\xbe\xc7\x8eoJK}\xde\xe8O\xf5\xdc', 'keyId': 7363326094432272385}}, 'operationTime': Timestamp(1728266808, 41), 'updatedExisting': True}, acknowledged=True)

In [17]:
#2. Write a query to update all movies where the genre is "Horror" and set their IMDb rating to 6.0 if it is currently null.
collection.update_many({"genres": "Horror", "imdb.rating": {"$exists": False}}, {"$set":
{"imdb.rating": 6.0}})


UpdateResult({'n': 0, 'electionId': ObjectId('7fffffff0000000000000121'), 'opTime': {'ts': Timestamp(1728266823, 9), 't': 289}, 'nModified': 0, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728266823, 9), 'signature': {'hash': b'k\x96\xee\xc2f<\xb9\n)\x94\xa9\xb6\x82\xaa\xec\x92\xbc`\xac\x93', 'keyId': 7363326094432272385}}, 'operationTime': Timestamp(1728266823, 9), 'updatedExisting': False}, acknowledged=True)

In [18]:
#3.Write a query to delete all movies that were released before the year 1950.
collection.delete_many({"year": {"$lt": 1950}})

DeleteResult({'n': 640, 'electionId': ObjectId('7fffffff0000000000000121'), 'opTime': {'ts': Timestamp(1728266832, 616), 't': 289}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728266832, 618), 'signature': {'hash': b'\x9f<]\xd1T\xfb\x9a_\xcdq\xccH\xd9\x8bQ\xe5\xdd\xc0\xb6N', 'keyId': 7363326094432272385}}, 'operationTime': Timestamp(1728266832, 616)}, acknowledged=True)

In [20]:
#Exercise 5: Text Search

#1.Ensure the title field is indexed for text search in MongoDB and write a query to search for movies that contain the word "love" in their title.
love_movies = collection.find(
    {"$text": {"$search": "love"}},
    {"_id": 0, "title": 1, "year": 1, "imdb.rating": 1} )

for movie in love_movies:
    print(movie)




{'imdb': {'rating': 5.5}, 'year': 2003, 'title': "It's All About Love"}
{'title': 'He Loves Me... He Loves Me Not', 'year': 2002, 'imdb': {'rating': 7.3}}
{'title': 'The Loving Story', 'year': 2011, 'imdb': {'rating': 7.5}}
{'title': 'Crazy Love', 'year': 1987, 'imdb': {'rating': 7.0}}
{'title': 'Endless Love', 'year': 1981, 'imdb': {'rating': 4.7}}
{'imdb': {'rating': 7.7}, 'year': 2003, 'title': 'Love Actually'}
{'title': 'Love, Money, Love', 'year': 2000, 'imdb': {'rating': 4.0}}
{'imdb': {'rating': 6.5}, 'year': 1999, 'title': 'For Love of the Game'}
{'imdb': {'rating': 6.3}, 'year': 2008, 'title': 'The Edge of Love'}
{'title': 'Love and Death', 'year': 1975, 'imdb': {'rating': 7.8}}
{'title': 'The Dark Matter of Love', 'year': 2012, 'imdb': {'rating': 7.4}}
{'imdb': {'rating': 6.4}, 'year': 2007, 'title': 'Love in the Time of Cholera'}
{'title': 'Love Sick', 'year': 2006, 'imdb': {'rating': 7.0}}
{'title': 'Women in Love', 'year': 1969, 'imdb': {'rating': 7.6}}
{'title': 'Love Sto

In [21]:
#2. Write a text search query to find movies where the word "war" appears in the title or plot, sorted by IMDb rating (Return only the top 5 results)
war_movies  = collection.find(
    {"$text": {"$search": "war"}},
    {"_id": 0, "title": 1}
).sort("imdb.rating", -1).limit(5)

for movie in war_movies:
    print(movie)

{'title': 'Day One'}
{'title': 'A War'}
{'title': 'American Hostage'}
{'title': 'The Last Season'}
{'title': 'Band of Brothers'}


In [22]:
#Exercise 6: Bonus Combining Multiple Queries

#1. Write a query to find all movies where the genre is "Action" and the IMDb rating is greater than 8 (Sort the results by release year in descending order).
action_high_rated_movies = collection.find(
    {"genres": "Action", "imdb.rating": {"$gt": 8}},
    {"_id": 0, "title": 1, "year": 1, "imdb.rating": 1, "genres": 1}
).sort("year", -1)

for movie in action_high_rated_movies:
    print(movie)



{'genres': ['Action', 'Thriller'], 'title': 'Liquidation', 'year': '2007è', 'imdb': {'rating': 8.6}}
{'genres': ['Animation', 'Action', 'Fantasy'], 'title': 'Hellsing Ultimate', 'year': '2006è2012', 'imdb': {'rating': 8.5}}
{'genres': ['Animation', 'Action', 'Fantasy'], 'title': 'Hellsing Ultimate', 'year': '2006è2012', 'imdb': {'rating': 8.5}}
{'genres': ['Action', 'Adventure', 'Drama'], 'title': 'Babylon 5', 'year': '1994è1998', 'imdb': {'rating': 8.2}}
{'genres': ['Short', 'Action', 'Comedy'], 'title': 'Kung Fury', 'year': 2015, 'imdb': {'rating': 8.3}}
{'genres': ['Documentary', 'Action', 'Adventure'], 'title': 'Meru', 'year': 2015, 'imdb': {'rating': 8.3}}
{'genres': ['Documentary', 'Action', 'History'], 'title': 'The Real Miyagi', 'year': 2015, 'imdb': {'rating': 9.3}}
{'imdb': {'rating': 8.3}, 'year': 2015, 'genres': ['Action', 'Adventure', 'Sci-Fi'], 'title': 'Mad Max: Fury Road'}
{'genres': ['Action', 'Crime', 'Drama'], 'title': "Let's Sin", 'year': 2014, 'imdb': {'rating': 8.

In [24]:
#2. Write a query to find all movies where the directors contain "Christopher Nolan" and the IMDb rating is greater than 8 (Return only the top 3 movies sorted by rating).
nolan_movies = collection.find(
    {"directors": "Christopher Nolan", "imdb.rating": {"$gt": 8}},
    {"_id": 0, "title": 1, "year": 1, "imdb.rating": 1, "directors": 1}
).sort("imdb.rating", -1).limit(3)

for movie in nolan_movies:
    print(movie)


{'imdb': {'rating': 9.0}, 'year': 2008, 'title': 'The Dark Knight', 'directors': ['Christopher Nolan']}
{'imdb': {'rating': 8.8}, 'year': 2010, 'title': 'Inception', 'directors': ['Christopher Nolan']}
{'imdb': {'rating': 8.7}, 'year': 2014, 'title': 'Interstellar', 'directors': ['Christopher Nolan']}
